# Import

In [1]:
from keras.models import load_model
from keras.models import Model
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_squared_error
import keras
import yfinance as yf
from matplotlib.backends.backend_pdf import PdfPages
import pickle
import json
from scipy.optimize import minimize
from datetime import datetime, timedelta

# read profit

In [2]:
json_file_path = 'settings/profit_statistical.json'

try:
    # read JSON file
    with open(json_file_path, 'r') as json_file:
        data = json.load(json_file)
        profit = data['profit']
except FileNotFoundError:
    # If the file can not be found, set the value of new_portfolio_value to 0
    profit = 0

print(f'Profit: {profit}')

Profit: 3.0436021743138326


# add dividend

In [3]:
excel_file_path_div = 'settings/Dividend2023.xlsx'
div = pd.read_excel(excel_file_path_div)
print(div)
# Divide all values by 12 in the column "Dividend"
div['Dividend'] = div['Dividend'] / 12

print(div)

# Set symbol column as inde
div.set_index('Share', inplace=True)

      Share  Dividend
0    ALV.DE     11.40
1    DBK.DE      0.30
2   VOW3.DE     27.82
3    BMW.DE      8.50
4    ADS.DE      0.70
5    BEI.DE      0.70
6    DTE.SG      0.70
7    SAP.DE      2.05
8   1COV.DE      0.00
9    BAS.DE      3.40
10  EOAN.DE      0.51
11   RWE.DE      0.90
      Share  Dividend
0    ALV.DE  0.950000
1    DBK.DE  0.025000
2   VOW3.DE  2.318333
3    BMW.DE  0.708333
4    ADS.DE  0.058333
5    BEI.DE  0.058333
6    DTE.SG  0.058333
7    SAP.DE  0.170833
8   1COV.DE  0.000000
9    BAS.DE  0.283333
10  EOAN.DE  0.042500
11   RWE.DE  0.075000


# Download stock data

In [4]:
def download_stock_data(stock_symbols, start_date, end_date, output_file):
    with open(output_file, 'w') as f:
        # write column headings
        f.write('Date,Open,High,Low,Close,Volume\n')
        
        #for symbol in stock_symbols:
            # load share data for the symbol (Share name) and the given time period
        stock_data = yf.download(symbol, start=start_date, end=end_date)
        stock_data = stock_data[['Open', 'High', 'Low', 'Close', 'Volume']]  # Auswahl der gewünschten Spalten
        stock_data.to_csv(f, header=False)  # Schreiben der Daten in die Datei

In [5]:
excel_file_path = 'settings/lastSequenceForPrediction.xlsx'

# store index of the excel file in this txt file
index_file_path = 'settings/lastSequenceForPrediction.txt'

# read index from the from the index file
try:
    with open(index_file_path, 'r') as index_file:
        current_row_index = int(index_file.read().strip())
except FileNotFoundError:
    current_row_index = 0

# read excel file
df_lastSeq = pd.read_excel(excel_file_path)

# Check, if index is out of bounds
if current_row_index >= len(df_lastSeq):
    print("Es gibt keine weiteren Zeilen in der Excel-Tabelle.")
else:
    # extract data from the actual row
    start_date_lastSeq = df_lastSeq.loc[current_row_index, 'start_date']
    end_date_lastSeq = df_lastSeq.loc[current_row_index, 'end_date']

    # Output of the actual data 
    print(f'Startdatum: {start_date_lastSeq}, Enddatum: {end_date_lastSeq}')

Startdatum: 2023-10-01, Enddatum: 2024-01-31


In [6]:
# Enter symbols
stock_symbols = ['ALV.DE', 'DBK.DE', 'VOW3.DE', 'BMW.DE', 'ADS.DE', 'BEI.DE', 'DTE.SG', 'SAP.DE', '1COV.DE', 'BAS.DE', 'EOAN.DE', 'RWE.DE']
start_date = '2020-01-01'
end_date = end_date_lastSeq

# load and store the data for every share
for symbol in stock_symbols:
    output_file = f'stock_data_{symbol}.csv'
    download_stock_data(symbol, start_date, end_date, output_file)

    # read CSV and select select the desired columns
    df = pd.read_csv(output_file, usecols=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])
    
    # Search CSV file for empty lines and remove them
    with open(output_file, 'r') as file:
        lines = file.readlines()

    # filter the empty lines
    lines = [line.strip() for line in lines if line.strip()]

    # Overwrite the file with the adjusted rows
    with open(output_file, 'w') as file:
        file.write('\n'.join(lines))

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [7]:
# store the data frames for different symbols in the dictionary
dfPre = {}

for symbol in stock_symbols:
    # load CSV
    dfPre[symbol] = pd.read_csv(f'stock_data_{symbol}.csv')

    # Drop empty rows
    dfPre[symbol].dropna(inplace=True)

    # transform date to index
    dfPre[symbol].set_index('Date', inplace=True)

    # Check the number of rows
    num_rows = dfPre[symbol].shape[0]

In [8]:
# Check missing values
missing_values_beforePre = {symbol: dfPre[symbol].isnull().values.any() for symbol in stock_symbols}

# missing values are replaced with teh mean of the row before and after the actual row
for symbol in stock_symbols:
    for column in dfPre[symbol].columns:
        missing_valuesPre = dfPre[symbol][column].isnull()
        dfPre[symbol].loc[missing_valuesPre, column] = (dfPre[symbol][column].shift() + dfPre[symbol][column].shift(-1)) / 2

# Check if there are still any missing values 
missing_values_afterPre = {symbol: dfPre[symbol].isnull().values.any() for symbol in stock_symbols}

# Output of the missing values before and after the Treatment
for symbol in stock_symbols:
    print(f"Fehlende Werte vor der Behandlung für {symbol} gefunden:", missing_values_beforePre[symbol])
    print(f"Fehlende Werte nach der Behandlung für {symbol} gefunden:", missing_values_afterPre[symbol])

# Output of the length of the data frame for all symbols
for symbol in stock_symbols:
    print(f"Länge des Datensatzes für {symbol}:", len(dfPre[symbol]))

Fehlende Werte vor der Behandlung für ALV.DE gefunden: False
Fehlende Werte nach der Behandlung für ALV.DE gefunden: False
Fehlende Werte vor der Behandlung für DBK.DE gefunden: False
Fehlende Werte nach der Behandlung für DBK.DE gefunden: False
Fehlende Werte vor der Behandlung für VOW3.DE gefunden: False
Fehlende Werte nach der Behandlung für VOW3.DE gefunden: False
Fehlende Werte vor der Behandlung für BMW.DE gefunden: False
Fehlende Werte nach der Behandlung für BMW.DE gefunden: False
Fehlende Werte vor der Behandlung für ADS.DE gefunden: False
Fehlende Werte nach der Behandlung für ADS.DE gefunden: False
Fehlende Werte vor der Behandlung für BEI.DE gefunden: False
Fehlende Werte nach der Behandlung für BEI.DE gefunden: False
Fehlende Werte vor der Behandlung für DTE.SG gefunden: False
Fehlende Werte nach der Behandlung für DTE.SG gefunden: False
Fehlende Werte vor der Behandlung für SAP.DE gefunden: False
Fehlende Werte nach der Behandlung für SAP.DE gefunden: False
Fehlende Werte

# Read close values at the begin of the time period

In [9]:
# Definition of the time period
# convert in to datetime objekt to substract days for the last day before this period
start_date_obj = datetime.strptime(end_date_lastSeq, '%Y-%m-%d')
end_date_obj = datetime.strptime(end_date_lastSeq, '%Y-%m-%d')
# Substract 5 days for the new start_date and 1 day for the new end_date
# In the next block the last date will be extracted 
new_start_date = start_date_obj - timedelta(days=5)
new_end_date = end_date_obj

print(new_start_date)
print(new_end_date)

# Load and store the data for every symbol (share)
for symbol in stock_symbols:
    output_file = f'stock_data_begin{symbol}.csv'
    download_stock_data(symbol, new_start_date, new_end_date, output_file)

    # Search and delete empty rows in the CSV file
    with open(output_file, 'r') as file:
        lines = file.readlines()

    # Filter the empty rows
    lines = [line.strip() for line in lines if line.strip()]

    # Overwrite the file with the cleaned data
    with open(output_file, 'w') as file:
        file.write('\n'.join(lines))


[*********************100%%**********************]  1 of 1 completed

2024-01-26 00:00:00
2024-01-31 00:00:00



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [10]:
# create data frame for the collected data 
combined_data_begin = pd.DataFrame(columns=['Symbol', 'Close_Value'])

# load and store the data for every symbol (share)
for symbol in stock_symbols:
    output_file = f'stock_data_begin{symbol}.csv'
    # read CSV file 
    df = pd.read_csv(output_file)

    # Select the last row only
    last_row = df.tail(1)

     # Write symbol and close value to the data frame
    data = {'Symbol': symbol, 'Close_Value': last_row['Close'].iloc[0]}
    combined_data_begin = pd.concat([combined_data_begin, pd.DataFrame(data, index=[0])], ignore_index=True)
    
# Add a row for the risk-free interest rate with the specified value
risk_free_rate = 4 # Input in percentage
risk_free_rate = risk_free_rate / 12 # Risk free rate per month
risk_free_data = {'Symbol': 'Risk_Free', 'Close_Value': 100}
combined_data_begin = combined_data_begin.append(risk_free_data, ignore_index=True)

print(combined_data_begin)

       Symbol  Close_Value
0      ALV.DE   247.600006
1      DBK.DE    12.100000
2     VOW3.DE   116.639999
3      BMW.DE    95.000000
4      ADS.DE   179.619995
5      BEI.DE   136.000000
6      DTE.SG    22.840000
7      SAP.DE   162.419998
8     1COV.DE    49.209999
9      BAS.DE    44.470001
10    EOAN.DE    12.420000
11     RWE.DE    34.299999
12  Risk_Free   100.000000


C:\Users\Samuel\AppData\Local\Temp\ipykernel_13792\18960333.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_data_begin = combined_data_begin.append(risk_free_data, ignore_index=True)


# load share value from 2019

In [11]:
excel_file_path = 'settings/actualMonth_startEnd.xlsx'

# Path to file, in which the index is stored
index_file_path = 'settings/actualMonthIndex.txt'

# Read the current index from the index file
try:
    with open(index_file_path, 'r') as index_file:
        current_row_index = int(index_file.read().strip())
except FileNotFoundError:
    current_row_index = 0
print(current_row_index)
# read excel file
df = pd.read_excel(excel_file_path)

# Check, if the index is out of bounds
if current_row_index >= len(df):
    print("Es gibt keine weiteren Zeilen in der Excel-Tabelle.")
else:
    # extract the data from the current row
    start_date = df.loc[current_row_index, 'start_date']
    end_date = df.loc[current_row_index, 'end_date']

    # Output of the current data
    print(f'Startdatum: {start_date}, Enddatum: {end_date}')

    # Updating the index in the index file for the next call
    with open(index_file_path, 'w') as index_file:
        index_file.write(str(current_row_index + 1))

13
Startdatum: 2024-02-01, Enddatum: 2024-02-28


In [12]:
start_date = start_date
end_date = end_date
print(f'Startdatum: {start_date}, Enddatum: {end_date}')
# Load and store the symbols for every symbol (share)
for symbol in stock_symbols:
    output_file = f'stock_data_{symbol}_2019.csv'
    download_stock_data(symbol, start_date, end_date, output_file)

    # search and delete for empty rows in the CSV 
    with open(output_file, 'r') as file:
        lines = file.readlines()

    # Filter the empty rows
    lines = [line.strip() for line in lines if line.strip()]

    # Overwrite the file with the cleaned rows
    with open(output_file, 'w') as file:
        file.write('\n'.join(lines))


[*********************100%%**********************]  1 of 1 completed


Startdatum: 2024-02-01, Enddatum: 2024-02-28


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [13]:
# list for the las close value of the shares
last_close_values = []

# load for each symbol
for symbol in stock_symbols:
    # file name of the csv data
    csv_file = f'stock_data_{symbol}_2019.csv'
    
    # load CSV data
    df = pd.read_csv(csv_file)
    
    # set date as index
    df.set_index('Date', inplace=True)
    
    # collect just the last close value and write it to the list
    last_close_value = df['Close'].iloc[-1]
    last_close_values.append((symbol, last_close_value))
print(last_close_values[0][1])

# Add a row for the risk-free interest rate with a value of 1
last_close_values.append(('Risk_Free', 100 + risk_free_rate))

# print the list
print("Liste der letzten Close-Werte:")
for symbol, last_close_value in last_close_values:
    print(f"{symbol}: {last_close_value}")

248.0
Liste der letzten Close-Werte:
ALV.DE: 248.0
DBK.DE: 12.383999824523926
VOW3.DE: 124.08000183105467
BMW.DE: 107.68000030517578
ADS.DE: 188.16000366210935
BEI.DE: 139.10000610351562
DTE.SG: 22.15999984741211
SAP.DE: 174.8000030517578
1COV.DE: 49.959999084472656
BAS.DE: 47.45500183105469
EOAN.DE: 11.699999809265137
RWE.DE: 31.01000022888184
Risk_Free: 100.33333333333333


# Split into blocks of 20 (20 business days per month)

In [14]:
# Dictionary for the splitted data
df_split = {}

# Quantity of the blocks per symbol
block_size = 20

for symbol, df in dfPre.items():
    # Quantity of the rows in the data frame
    num_rows = len(df)

    # Split the data frame in to blocks
    split_dfs = np.array_split(df, num_rows // block_size)

    # Add the solited blocks to the dictionary
    df_split[symbol] = split_dfs

# Output result
for symbol, split_dfs in df_split.items():
    print(f"Symbol: {symbol}")
    for i, df_block in enumerate(split_dfs):
        print(f"Block {i+1}:")
        print(df_block)
        print()

Symbol: ALV.DE
Block 1:
                  Open        High         Low       Close   Volume
Date                                                               
2020-01-02  218.600006  222.300003  218.399994  221.500000   846800
2020-01-03  220.100006  220.399994  217.649994  219.050003   997575
2020-01-06  217.149994  217.800003  215.199997  217.699997   842331
2020-01-07  218.500000  220.149994  217.850006  218.199997   905810
2020-01-08  217.199997  219.199997  215.850006  218.300003  1006392
2020-01-09  220.149994  221.350006  219.850006  220.699997   998027
2020-01-10  221.500000  221.949997  220.250000  220.250000   762695
2020-01-13  221.250000  221.449997  219.250000  219.649994   698396
2020-01-14  219.699997  219.800003  217.949997  218.850006   827509
2020-01-15  218.649994  219.000000  216.149994  217.199997  1105804
2020-01-16  217.800003  218.649994  216.600006  217.350006   816269
2020-01-17  217.850006  219.649994  217.449997  219.649994  1250857
2020-01-20  219.449997  

              Open    High     Low   Close    Volume
Date                                                
2021-11-23  10.800  11.188  10.786  11.180  14866248
2021-11-24  11.150  11.548  11.098  11.450  16356050
2021-11-25  11.462  11.572  11.404  11.570   8793212
2021-11-26  10.920  11.038  10.700  10.700  22453286
2021-11-29  10.936  10.966  10.646  10.678  13389305
2021-11-30  10.500  10.790  10.410  10.660  14952686
2021-12-01  10.792  11.004  10.792  10.886   9992083
2021-12-02  10.732  10.978  10.698  10.922   9779658
2021-12-03  11.040  11.094  10.726  10.742  10084663
2021-12-06  10.980  11.216  10.936  11.174  11810118
2021-12-07  11.174  11.174  11.174  11.174         0
2021-12-08  11.384  11.480  11.138  11.408   8235012
2021-12-09  11.270  11.296  11.000  11.022   9418780
2021-12-10  10.960  11.044  10.890  10.920   6477812
2021-12-13  10.980  11.026  10.788  10.806   5573870
2021-12-14  10.870  11.054  10.808  10.930   7452098
2021-12-15  10.982  11.014  10.802  10.836   5

                 Open       High        Low      Close   Volume
Date                                                           
2020-01-02  73.139999  74.629997  73.059998  74.220001  1232319
2020-01-03  73.730003  73.830002  72.330002  73.320000  1153245
2020-01-06  72.830002  73.050003  72.040001  73.050003  1039192
2020-01-07  73.599998  74.320000  73.339996  74.220001  1302124
2020-01-08  73.940002  74.500000  73.750000  74.410004   941807
2020-01-09  75.099998  75.349998  74.360001  74.360001  1391131
2020-01-10  74.489998  74.959999  74.220001  74.570000  1102138
2020-01-13  74.699997  74.809998  73.389999  73.629997   884147
2020-01-14  73.739998  73.820000  73.059998  73.300003   979352
2020-01-15  73.089996  73.269997  71.779999  72.190002  1607651
2020-01-16  72.190002  72.459999  70.570000  71.169998  1730300
2020-01-17  71.690002  71.949997  70.910004  71.059998  2112839
2020-01-20  70.919998  71.860001  70.820000  71.639999   837922
2020-01-21  71.389999  72.000000  70.930

                 Open       High        Low      Close   Volume
Date                                                           
2020-06-26  57.810001  57.860001  55.490002  55.770000  1953166
2020-06-29  55.540001  56.840000  55.340000  56.709999  1859116
2020-06-30  56.799999  57.119999  55.720001  56.820000  2049620
2020-07-01  56.009998  57.060001  55.639999  56.200001  1618381
2020-07-02  56.939999  58.880001  56.639999  57.950001  2018441
2020-07-03  58.209999  58.549999  57.009998  57.259998   903202
2020-07-06  58.950001  59.680000  57.730000  58.459999  1512797
2020-07-07  58.310001  59.279999  57.759998  58.650002  1505768
2020-07-08  58.270000  59.189999  57.860001  58.020000  1362427
2020-07-09  58.610001  58.990002  57.279999  57.439999  1231470
2020-07-10  57.250000  58.700001  56.820000  58.689999  1532862
2020-07-13  59.599998  59.930000  58.310001  58.799999  1073725
2020-07-14  57.919998  58.509998  57.340000  57.930000  1371809
2020-07-15  58.330002  59.110001  58.049

                  Open        High         Low       Close  Volume
Date                                                              
2020-05-28   94.279999   95.540001   93.300003   95.540001  447751
2020-05-29   95.139999   95.779999   93.699997   94.260002  619600
2020-06-02   95.739998   96.639999   94.720001   96.180000  510599
2020-06-03   96.279999   99.120003   96.080002   98.959999  666417
2020-06-04   99.239998  101.500000   99.080002   99.500000  706431
2020-06-05   99.599998   99.860001   98.459999   99.080002  558864
2020-06-08   98.980003   99.599998   97.139999   98.959999  584247
2020-06-09   98.900002  100.449997   98.320000   99.959999  652640
2020-06-10  100.099998  100.849998   99.360001   99.839996  471612
2020-06-11   99.139999   99.699997   98.440002   98.440002  585708
2020-06-12   98.239998  100.099998   97.900002   98.239998  489493
2020-06-15   96.500000   97.440002   95.519997   96.379997  493783
2020-06-16   96.980003  100.150002   96.540001   99.900002  55

...

                  Open        High         Low       Close   Volume
Date                                                               
2021-08-03  121.239998  122.959999  120.720001  121.900002  1416262
2021-08-04  122.800003  123.580002  122.059998  123.379997  1584403
2021-08-05  124.000000  125.940002  123.839996  125.779999  1624362
2021-08-06  125.639999  125.919998  124.339996  124.839996  1218718
2021-08-09  124.339996  124.839996  123.660004  124.680000  1165047
2021-08-10  124.720001  126.680000  124.239998  124.239998  1113410
2021-08-11  124.180000  125.099998  123.580002  123.760002  1445962
2021-08-12  122.980003  125.940002  122.440002  125.900002  1512253
2021-08-13  125.500000  126.279999  125.139999  125.480003  1152899
2021-08-16  124.519997  125.720001  124.199997  125.660004  1185314
2021-08-17  125.239998  127.339996  124.779999  127.099998  1522253
2021-08-18  126.779999  127.379997  126.660004  126.919998  1194331
2021-08-19  125.160004  125.919998  124.080002  

                 Open       High        Low      Close   Volume
Date                                                           
2021-03-11  59.919998  61.279999  59.639999  60.200001   751990
2021-03-12  59.919998  60.000000  59.020000  59.139999   761729
2021-03-15  59.000000  59.119999  56.900002  57.279999   990220
2021-03-16  57.419998  57.880001  56.540001  56.959999   998584
2021-03-17  56.959999  57.759998  56.619999  57.680000   963094
2021-03-18  58.060001  59.040001  57.900002  58.700001  1038278
2021-03-19  58.119999  58.180000  55.180000  55.419998  2609509
2021-03-22  55.139999  55.299999  53.919998  54.200001  1405685
2021-03-23  54.000000  54.139999  52.840000  53.700001  1390888
2021-03-24  53.380001  54.520000  53.200001  54.119999  1327063
2021-03-25  53.660000  54.139999  52.700001  54.139999  1121795
2021-03-26  54.779999  56.720001  54.779999  56.360001  1801582
2021-03-29  56.580002  57.000000  55.779999  56.439999  1152971
2021-03-30  56.919998  58.299999  56.740

                 Open       High        Low      Close    Volume
Date                                                            
2023-02-21  53.110001  53.279999  52.029999  52.490002   2003891
2023-02-22  52.160000  52.410000  51.400002  52.360001   2447350
2023-02-23  52.680000  52.779999  51.980000  52.169998   2039899
2023-02-24  50.799999  50.950001  48.070000  48.070000  15255950
2023-02-27  48.105000  48.990002  47.814999  47.889999   4646830
2023-02-28  47.674999  48.654999  47.529999  48.474998   4673550
2023-03-01  48.910000  49.325001  48.290001  48.375000   3534679
2023-03-02  47.369999  48.435001  47.215000  48.325001   2886750
2023-03-03  48.540001  49.285000  48.535000  49.165001   2411864
2023-03-06  49.299999  49.320000  48.150002  48.255001   3025485
2023-03-07  48.220001  48.270000  47.849998  48.014999   2279063
2023-03-08  47.669998  48.509998  47.430000  48.285000   2605483
2023-03-09  48.380001  48.465000  47.820000  47.990002   1659878
2023-03-10  47.279999  47

              Open    High    Low   Close    Volume
Date                                               
2022-05-16   9.842   9.948  9.700   9.930   7617450
2022-05-17   9.938  10.080  9.884   9.914   6876842
2022-05-18  10.000  10.170  9.998  10.075   8355744
2022-05-19  10.010  10.040  9.918   9.956   6025609
2022-05-20   9.974  10.175  9.966  10.045   6868158
2022-05-23  10.185  10.195  9.962  10.105   5242997
2022-05-24   9.918   9.950  9.736   9.780   8292247
2022-05-25   9.780   9.780  9.780   9.780         0
2022-05-26   9.926   9.958  9.814   9.880   5795860
2022-05-27   9.880   9.880  9.880   9.880         0
2022-05-30   9.880   9.880  9.880   9.880         0
2022-05-31   9.582   9.612  9.430   9.484  12425326
2022-06-01   9.508   9.634  9.452   9.510   8025692
2022-06-02   9.510   9.510  9.510   9.510         0
2022-06-03   9.510   9.510  9.510   9.510         0
2022-06-06   9.700   9.730  9.572   9.646   3278733
2022-06-07   9.600   9.672  9.572   9.668   3497980
2022-06-08  

                 Open       High        Low      Close   Volume
Date                                                           
2022-07-11  35.459999  36.820000  35.060001  36.490002  2193174
2022-07-12  36.490002  36.490002  36.490002  36.490002        0
2022-07-13  36.279999  36.320000  35.810001  36.160000  1159359
2022-07-14  36.180000  36.330002  35.110001  35.560001  2018485
2022-07-15  35.849998  36.959999  35.790001  36.820000  2561174
2022-07-18  36.939999  37.320000  36.480000  37.110001  2295583
2022-07-19  37.060001  38.950001  36.869999  38.590000  2553486
2022-07-20  38.560001  38.770000  37.880001  38.259998  1783490
2022-07-21  38.599998  38.759998  37.110001  37.250000  1511566
2022-07-22  37.160000  38.000000  37.099998  37.520000  1584364
2022-07-25  37.150002  38.560001  37.060001  37.840000  1435411
2022-07-26  38.099998  38.520000  37.619999  38.150002  1771819
2022-07-27  38.189999  39.889999  38.020000  39.160000  2736497
2022-07-28  39.320000  39.970001  38.349

# Calculate the return for every month

In [15]:
# Dictionary for the increase/decrease per symbol
change_per_symbol = {}

for symbol, split_dfs in df_split.items():
    changes = []

    # Iterate over every block of the current symbol
    for df_block in split_dfs:
        # Extract the first and the lat close value of the current block
        first_close = df_block.iloc[0]['Close']
        last_close = df_block.iloc[-1]['Close']

        # Calculate the change of the close value
        change = (last_close - first_close + div.loc[symbol, 'Dividend']) / first_close * 100  # in Prozent
        print(symbol)
        print(div.loc[symbol, 'Dividend'])
        changes.append(change)

    # Save the Change for the current symbol
    change_per_symbol[symbol] = changes

# Output result
for symbol, changes in change_per_symbol.items():
    print(f"Symbol: {symbol}")
    for i, change in enumerate(changes):
        print(f"Block {i+1}: {change:.2f}%")
    print()


ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.9500000000000001
ALV.DE
0.950

0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0.05833333333333333
DTE.SG
0

# Calculate the mean return over the period of 4 years

In [16]:
# Dictionary for the mean return per symbol
average_return_per_symbol = {}

for symbol, changes in change_per_symbol.items():
    # Calculate the mean return for the current symbol
    average_return = sum(changes) / len(changes)
    average_return_per_symbol[symbol] = average_return

# Add Risk_Free symbol with risk_free_rate average return
average_return_per_symbol['Risk_Free'] = risk_free_rate    

# Output result
for symbol, avg_return in average_return_per_symbol.items():
    print(f"Durchschnittliche Rendite für {symbol}: {avg_return:.2f}%")
    
# Convert mean return to DataFrame
average_return_per_symbol_df = pd.DataFrame(list(average_return_per_symbol.items()), columns=['Symbol', 'Average_Return'])
print(average_return_per_symbol_df)


Durchschnittliche Rendite für ALV.DE: 0.95%
Durchschnittliche Rendite für DBK.DE: 1.83%
Durchschnittliche Rendite für VOW3.DE: 1.69%
Durchschnittliche Rendite für BMW.DE: 2.03%
Durchschnittliche Rendite für ADS.DE: -0.27%
Durchschnittliche Rendite für BEI.DE: 0.39%
Durchschnittliche Rendite für DTE.SG: 1.19%
Durchschnittliche Rendite für SAP.DE: 0.99%
Durchschnittliche Rendite für 1COV.DE: 0.45%
Durchschnittliche Rendite für BAS.DE: 0.22%
Durchschnittliche Rendite für EOAN.DE: 0.71%
Durchschnittliche Rendite für RWE.DE: 0.46%
Durchschnittliche Rendite für Risk_Free: 0.33%
       Symbol  Average_Return
0      ALV.DE        0.950716
1      DBK.DE        1.834955
2     VOW3.DE        1.694489
3      BMW.DE        2.028557
4      ADS.DE       -0.266928
5      BEI.DE        0.387227
6      DTE.SG        1.186782
7      SAP.DE        0.985046
8     1COV.DE        0.446011
9      BAS.DE        0.215875
10    EOAN.DE        0.711023
11     RWE.DE        0.460389
12  Risk_Free        0.333333


In [17]:
# Dictionary for the standard deviation per symbol
standard_deviation_per_symbol = {}

for symbol, changes in change_per_symbol.items():
    
    # Calculate the difference to the power of 2
    squared_diff = [(change - average_return_per_symbol[symbol]) ** 2 for change in changes]
    
    # Sum up the differences
    sum_squared_diff = sum(squared_diff)

    # Calculate the standard deviation
    standard_deviation = np.sqrt(sum_squared_diff / (len(changes) - 1))
    
    # Save the standard deviation in the dictionary
    standard_deviation_per_symbol[symbol] = standard_deviation

# Add Risk_Free symbol with risk_free_rate average return
standard_deviation_per_symbol['Risk_Free'] = 0.0  

# Output result
for symbol, std_dev in standard_deviation_per_symbol.items():
    print(f"Standardabweichung für {symbol}: {std_dev:.2f}")
    
# Convert mean return to DataFrame
standard_deviation_per_symbol_df = pd.DataFrame(list(standard_deviation_per_symbol.items()), columns=['Symbol', 'Standard_Deviation'])
print(standard_deviation_per_symbol_df)

Standardabweichung für ALV.DE: 6.50
Standardabweichung für DBK.DE: 10.02
Standardabweichung für VOW3.DE: 9.70
Standardabweichung für BMW.DE: 8.14
Standardabweichung für ADS.DE: 8.74
Standardabweichung für BEI.DE: 3.97
Standardabweichung für DTE.SG: 4.48
Standardabweichung für SAP.DE: 7.47
Standardabweichung für 1COV.DE: 8.29
Standardabweichung für BAS.DE: 8.24
Standardabweichung für EOAN.DE: 6.19
Standardabweichung für RWE.DE: 8.12
Standardabweichung für Risk_Free: 0.00
       Symbol  Standard_Deviation
0      ALV.DE            6.496072
1      DBK.DE           10.017627
2     VOW3.DE            9.703508
3      BMW.DE            8.136622
4      ADS.DE            8.736580
5      BEI.DE            3.968538
6      DTE.SG            4.483129
7      SAP.DE            7.465593
8     1COV.DE            8.289448
9      BAS.DE            8.237326
10    EOAN.DE            6.190998
11     RWE.DE            8.116173
12  Risk_Free            0.000000


# Historic data from 2018 for the covarianz matrix

In [18]:
excel_file_path = 'settings/lastSequenceForPrediction.xlsx'

# path to the data, in which the index is stored
index_file_path = 'settings/lastSequenceForPrediction.txt'

# read the current index from the index file
try:
    with open(index_file_path, 'r') as index_file:
        current_row_index = int(index_file.read().strip())
except FileNotFoundError:
    current_row_index = 0

# read excel file
df_lastSeq = pd.read_excel(excel_file_path)

# Check, if index is out of bounds
if current_row_index >= len(df_lastSeq):
    print("Es gibt keine weiteren Zeilen in der Excel-Tabelle.")
else:
    # extract data from the current row
    start_date_lastSeq = df_lastSeq.loc[current_row_index, 'start_date']
    end_date_lastSeq = df_lastSeq.loc[current_row_index, 'end_date']
    
    # Output of the current data 
    print(f'Startdatum: {start_date_lastSeq}, Enddatum: {end_date_lastSeq}')

    # Update the index in the index file for the next call
    with open(index_file_path, 'w') as index_file:
        index_file.write(str(current_row_index + 1))

Startdatum: 2023-10-01, Enddatum: 2024-01-31


In [19]:
# Definition of the time period
start_date2018 = '2023-01-01'
end_date2018 = end_date_lastSeq
print(end_date2018)
# Load and store the data for every symbol (share)
for symbol in stock_symbols:
    output_file = f'stock_data_{symbol}_2018.csv'
    download_stock_data(symbol, start_date2018, end_date2018, output_file)

    # Search and delete empty rows in the CSV file
    with open(output_file, 'r') as file:
        lines = file.readlines()

    # Filter the empty rows
    lines = [line.strip() for line in lines if line.strip()]

    # Overwrite the file with the cleaned rows
    with open(output_file, 'w') as file:
        file.write('\n'.join(lines))


2024-01-31


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


# Combine historical data to one file

In [20]:
# Data frame for the combined data
combined_historical2018_data = pd.DataFrame()

# load and combine the data for each share
for symbol in stock_symbols:
    # file name for the csv file
    csv_file = f'stock_data_{symbol}_2018.csv'
    
    # read CSV file and set date to index
    df = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
    
    # change name of the close column
    df.rename(columns={'Close': symbol}, inplace=True)
    
    # write data to the combined file
    combined_historical2018_data = pd.concat([combined_historical2018_data, df[symbol]], axis=1)

# drop rows with NaN values
combined_historical2018_data.dropna(inplace=True)

# Initialize the initial and final values for "Risk_Free"
risk_free_initial = 100
risk_free_final = 100 + risk_free_rate

# Create a series for "Risk_Free" with increasing values
risk_free_values = np.linspace(risk_free_initial, risk_free_final, len(combined_historical2018_data))

# Add the "Risk_Free" series to the DataFrame
combined_historical2018_data['Risk_Free'] = risk_free_values

# create a CSV file for the data
combined_historical2018_data.to_csv('combined_historical2018_stock_data.csv')

print(combined_historical2018_data)

                ALV.DE  DBK.DE     VOW3.DE     BMW.DE      ADS.DE      BEI.DE  \
2023-01-02  203.050003  10.942  120.040001  85.800003  127.699997  107.150002   
2023-01-03  205.199997  11.112  122.059998  85.830002  131.880005  107.500000   
2023-01-04  211.500000  11.698  125.879997  87.879997  138.380005  108.800003   
2023-01-05  209.550003  11.494  127.120003  88.949997  138.539993  107.800003   
2023-01-06  211.800003  11.596  128.160004  89.529999  140.679993  108.300003   
...                ...     ...         ...        ...         ...         ...   
2024-01-24  248.500000  12.164  116.000000  93.550003  166.139999  132.399994   
2024-01-25  246.850006  12.080  114.839996  93.690002  175.679993  133.149994   
2024-01-26  248.100006  12.080  115.339996  94.769997  177.860001  134.149994   
2024-01-29  246.550003  11.976  115.959999  94.910004  178.000000  135.600006   
2024-01-30  247.600006  12.100  116.639999  95.000000  179.619995  136.000000   

               DTE.SG      

# Mean-Variance Optimization

In [21]:
def mean_variance_optimization(expected_returns, standard_deviations, covariance_matrix):
    n = len(expected_returns)
    initial_weights = np.array([1/n] * n)  # Start value of the weighs
    bounds = [(0, 1)] * n  # Border of the weighs (0-100% for every symbol)

    # Minimize the negative sharpe ratio
    def negative_sharpe(weights, expected_returns, standard_deviations, covariance_matrix):
        portfolio_return = np.dot(weights, expected_returns)
        portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))
        return -portfolio_return / portfolio_volatility

    result = minimize(negative_sharpe, initial_weights, args=(expected_returns, standard_deviations, covariance_matrix), bounds=bounds, constraints={'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    return result.x

In [22]:
def allocate_portfolio(historical_data, initial_capital, expected_returns_df, standard_deviations_df):
    # Calculate expected returns based on the predictions
    expected_returns = expected_returns_df['Average_Return'].values
    
    # Calculate standard deviations based on the predictions
    standard_deviations = standard_deviations_df['Standard_Deviation'].values
    
    # Calculate the covariance matrix of the returns
    covariance_matrix = historical_data.cov()
           
    print(covariance_matrix)
    
    # Perform mean-variance optimization to obtain optimal weights
    optimal_weights = mean_variance_optimization(expected_returns, standard_deviations, covariance_matrix)

    # Calculate the allocation of assets based on the optimal weights and the available capital
    asset_allocation = initial_capital * optimal_weights

    return asset_allocation

In [23]:
initial_capital = 1000 + profit # 1000€ start capital
portfolio_allocation = allocate_portfolio(combined_historical2018_data, initial_capital, average_return_per_symbol_df, standard_deviation_per_symbol_df)
print(portfolio_allocation)

               ALV.DE    DBK.DE    VOW3.DE     BMW.DE      ADS.DE     BEI.DE  \
ALV.DE     127.705335  7.764961 -44.897491 -20.482000   85.424112  58.590672   
DBK.DE       7.764961  1.022538   0.158764  -2.539568    0.015421   1.357643   
VOW3.DE    -44.897491  0.158764  78.349266  21.827532  -75.153445 -37.674591   
BMW.DE     -20.482000 -2.539568  21.827532  33.858066   22.497725   2.551926   
ADS.DE      85.424112  0.015421 -75.153445  22.497725  236.017905  75.766631   
BEI.DE      58.590672  1.357643 -37.674591   2.551926   75.766631  53.251501   
DTE.SG       6.772073  0.413774  -1.121148  -1.430682    0.567401   4.926163   
SAP.DE      99.160465  2.757432 -67.729010   1.814085  149.299507  78.774175   
1COV.DE     35.765029  1.458024 -29.679656  -2.320594   56.898672  18.089661   
BAS.DE      -4.920428  1.207553  19.783214  -0.207959  -22.369503 -10.851896   
EOAN.DE      4.465990 -0.037015  -2.888871   1.379042    8.140850   4.960813   
RWE.DE      -3.371612  0.396955  11.5757

In [24]:
# Initialisation of the portfolio
portfolio_value = 0
# initialize the list for the quantity of the shares, which have to be purchased
shares_to_buy_list = []
# Purchase the shares based on the allocations
for i, allocation in enumerate(portfolio_allocation):
    # Calculate the quantity of the shares, which should be purchased with this allocation
    shares_to_buy = (allocation) / combined_data_begin['Close_Value'][i]
    shares_to_buy_list.append(shares_to_buy)
    print(shares_to_buy)

    # Calculate the value of the stock of the purchased shares
    value_of_stock = shares_to_buy * combined_data_begin['Close_Value'][i]
    print(value_of_stock)
    
    # Add the value of the purchased share to the portfolio
    portfolio_value += value_of_stock
    print(portfolio_value)
    print("-----------------------------")

# Output of the total value of the portfolio at the end of the period to be predicted
print("Gesamtwert des Portfolios am Ende des ersten Monats von 2019:", portfolio_value)
print(shares_to_buy_list)

1.53471769400321e-11
3.799961104023682e-09
3.799961104023682e-09
-----------------------------
7.755457218506676e-10
9.384103530240294e-09
1.3184064634263976e-08
-----------------------------
0.06124540891194157
7.143664458107634
7.143664471291698
-----------------------------
3.2496057893923656e-07
3.087125499922747e-05
7.143695342546698
-----------------------------
2.8283921208196497e-11
5.080357789311171e-09
7.143695347627055
-----------------------------
9.923250237726685e-12
1.3495620323308293e-09
7.1436953489766175
-----------------------------
4.5881817685293536e-06
0.00010479407229331142
7.143800143048911
-----------------------------
2.739431330212066e-11
4.449384316369951e-09
7.143800147498295
-----------------------------
2.629864119294656e-07
1.2941561090277752e-05
7.143813089059385
-----------------------------
0.15796929762601614
7.024894858262553
14.168707947321938
-----------------------------
7.571061866331095e-06
9.403258895745836e-05
14.168801979910896
-------------

In [25]:
# Define portfolio value
new_portfolio_value = 0
# Loop over the indices of the two lists
for i in range(len(shares_to_buy_list)):
    # convert to float
    last_close_value = float(last_close_values[i][1])
    print(last_close_value)
    print(shares_to_buy_list[i])
    #Calculate the new value of the share and add the value to the list
    new_portfolio_value += (last_close_value * shares_to_buy_list[i])
    print(new_portfolio_value)
    print("----------------------------")

print("Wert des Portfolios:")
print(new_portfolio_value)

248.0
1.53471769400321e-11
3.806099881127961e-09
----------------------------
12.383999824523926
7.755457218506676e-10
1.341045796443691e-08
----------------------------
124.08000183105467
0.06124540891194157
7.59933046334786
----------------------------
107.68000030517578
3.2496057893923656e-07
7.599365455103099
----------------------------
188.16000366210935
2.8283921208196497e-11
7.599365460425002
----------------------------
139.10000610351562
9.923250237726685e-12
7.599365461805326
----------------------------
22.15999984741211
4.5881817685293536e-06
7.599467135912617
----------------------------
174.8000030517578
2.739431330212066e-11
7.599467140701143
----------------------------
49.959999084472656
2.629864119294656e-07
7.599480279502042
----------------------------
47.45500183105469
0.15796929762601614
15.09591358759506
----------------------------
11.699999809265137
7.571061866331095e-06
15.096002169017453
----------------------------
31.01000022888184
2.926201041914449e-06
15

In [26]:
# just the profit, to store it
profit = new_portfolio_value - 1000

# path to JSON file
json_file_path = 'settings/profit_statistical.json'

# Create Dictionary with the value of new_portfolio_value
data = {'profit': profit}

# Write JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(data, json_file)

print(f'Wert von profit wurde erfolgreich in {json_file_path} gespeichert.')

Wert von profit wurde erfolgreich in settings/profit_statistical.json gespeichert.
